
### runs on stenv3

In [9]:
import os
import sys
import glob
import scanpy as sc
import numpy as np

### Reads in all fastq file names on gcp

In [4]:
%%capture cap_out --no-stderr
!gsutil ls gs://fc-17b85933-14fb-4c4d-937e-e928f8ea53b1/fastq/ | cut -d '/' -f 5 
fastqs = [i for i in cap_out.stdout.split("\r\n")]
fastqs_samples = [j[0]+"_"+j[-1] for j in [i.split("_")[0:2] for i in fastqs]]

In [5]:
len(fastqs_samples)-1

790

### Reads in expression file

In [10]:
# Load Lambda pmean df
path = '/home/sanjavickovic/data/st_data'

# Read expression file
filename = os.path.join(path, 'anndata_colons_norm_metagenes_submodules_celltypes_kegg_degenes_pheno.h5ad')  
st_splotch_pd = sc.read_h5ad(filename)

In [17]:
len(np.unique(st_splotch_pd.obs.Age))*len(np.unique(st_splotch_pd.obs.Region))

33

### Checks if any extra files deposited on GCP and deletes those

In [9]:
for sam in np.unique(fastqs_samples[0:len(fastqs_samples)-1]):
    if not sam in np.unique(list(st_splotch_pd.obs['sample'])):
        #print("not ok")
        print(sam)
        dels = [x for x in fastqs if sam in x]
        if len(dels) > 1:
            fname_r1 = 'gs://fc-17b85933-14fb-4c4d-937e-e928f8ea53b1/fastq/'+dels[0]
            fname_r2 = 'gs://fc-17b85933-14fb-4c4d-937e-e928f8ea53b1/fastq/'+dels[1]
        else:
            fname_r1 = 'gs://fc-17b85933-14fb-4c4d-937e-e928f8ea53b1/fastq/'+dels[0]
        !gsutil rm $fname_r1
        !gsutil rm $fname_r2

In [10]:
%%capture cap_out --no-stderr
!gsutil ls gs://fc-17b85933-14fb-4c4d-937e-e928f8ea53b1/fastq/ | cut -d '/' -f 5 
fastqs = [i for i in cap_out.stdout.split("\r\n")]
fastqs_samples = [j[0]+"_"+j[-1] for j in [i.split("_")[0:2] for i in fastqs]]

### Checks if any files are missing on GCP

In [11]:
for sam in np.unique(list(st_splotch_pd.obs['sample'])):
    dels = [x for x in fastqs if sam in x]
    if len(dels) == 1:
        print(sam)
        
# copy any files outputed here to gcp for storage

In [12]:
%%capture cap_out --no-stderr
!gsutil ls gs://fc-17b85933-14fb-4c4d-937e-e928f8ea53b1/fastq/ | cut -d '/' -f 5 
fastqs = [i for i in cap_out.stdout.split("\r\n")]
fastqs_samples = [j[0]+"_"+j[-1] for j in [i.split("_")[0:2] for i in fastqs]]

In [13]:
assert((len(np.unique(fastqs_samples))-1) == (len(fastqs)-1)/2)